## 第5章：大規模言語モデル
この章では、大規模言語モデル(LLM)の利用し、様々なタスクに取り組む。大規模言語モデルをプログラムからAPI経由で呼び出すことを想定しており、そのAPIの利用で費用が発生する可能性があることに留意せよ。

### 40.Zero-Shot推論
以下の問題の回答を作成せよ。ただし、解答生成はZero-Shot推論とせよ。



９世紀に活躍した人物に関係するできごとについて述べた次のア〜ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
    query = ("9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。"
             "ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。"
             "イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。"
             "ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。"
             ) # ← ここをいろいろと変えて触ってみてください
    

    start = time.time()
    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    
    # 入力をプロンプトに挿入してtokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 推論を実行
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # repetition_penalty=1.1,
    )
        
    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    # 結果を表示
    print(response.replace("\n", " "))
    
    end = time.time()
    print(f"推論にかかった時間 : {str(end-start)} [s]")


if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [02:18<00:00, 34.51s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


正しい年代順は次のとおりです。  1. **イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。** （809年） 2. **ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。** （842年） 3. **ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。** （903年）    **解説:**  * イは嵯峨天皇の即位後すぐに起こった出来事です。 * ウは承和の変という事件の後、藤原良房が台頭した出来事です。 * アは菅原道真が追放された出来事で、時平が政界に台頭した後の出来事です。
推論にかかった時間 : 18.266639232635498 [s]


### 41.Few-Shot推論
以下の問題と回答を与え、問題40で示した質問の回答をfew-shot推論（この場合は4-shot推論）で生成せよ。

日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答: ウ→イ→ア

江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

解答: ウ→ア→イ

中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

解答: イ→ア→ウ

加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

解答: ウ→ア→イ

In [10]:
x = [{1,2},{3,4},{5,6}]
print(*x)
print(x)

{1, 2} {3, 4} {5, 6}
[{1, 2}, {3, 4}, {5, 6}]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の歴史学者です。"
    few_shot  = [(
        "日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。\n\n" 
    "ア　府知事・県令からなる地方官会議が設置された。\n" 
    "イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。\n" 
    "ウ　すべての藩主が，天皇に領地と領民を返還した。\n" 
    "解答: ウ→イ→ア\n\n"),
    ("江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。\n\n"
     "ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。\n"
     "イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。\n"
     "ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。\n"
     "解答: ウ→ア→イ\n\n"),
    ("中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。\n\n"
     "ア　アヘン戦争がおこり，清がイギリスに敗北した。\n"
     "イ　異国船打払令が出され，外国船を撃退することが命じられた。\n"
     "ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。\n"
     "解答: イ→ア→ウ\n\n"),
    ("加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。\n\n"
     "ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。\n"
     "イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。\n"
     "ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。\n"
     "解答: ウ→ア→イ\n\n")
    ]
    query = (
             "9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。\n\n"
             "ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。\n"
             "イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。\n"
             "ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。\n"
             "解答:"
             ) # ← ここをいろいろと変えて触ってみてください
    

    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "examples", "content": few_shot},
        {"role": "user", "content": query},
    ]
    
    # 入力をプロンプトに挿入してtokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 推論を実行
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # repetition_penalty=1.1,
    )
        
    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    # 結果を表示
    print(response.replace("\n", " "))
    


if __name__ == '__main__':
    main()


`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]


解答: イ→ウ→ア  **解説:**  * **イ** 嵯峨天皇が藤原冬嗣らを蔵人頭に任命したのは、809年のことです。 * **ウ** 承和の変は842年に起こり、その後、藤原良房は北家の優位を確立しました。 * **ア** 藤原時平が菅原道真を追放したのは、901年のことです。


In [1]:
temp = ['A', 'B']
x = 'C'
while x not in temp:
    print('あ')
    x = 'A'

あ


### 42.多肢選択問題の正解率
JMMLUのいずれかの科目を大規模言語モデルに解答させ、その正解率を求めよ。

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def get_data(path):
    data = pd.read_csv(path, header=None)
    data.columns = ['question', 'A', 'B', 'C', 'D', 'ans']
    return data

def main():

    data = get_data('high_school_biology.csv')

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の物理学者です。以下の質問の答えとなる選択肢をA,B,C,Dの中から1つ選択してください。この時、あなたの解答として許されるのは「A」,「B」,「C」,「D」などの選択肢1文字です。その他の解説などは出力しないでください。}"
    query = ("{question}\n\n"
             "A:{A}\n"
             "B:{B}\n"
             "C:{C}\n"
             "D:{D}\n"
             "解答:"
             ) # ← ここをいろいろと変えて触ってみてください
    error_query = ("あなたの解答{llmans}は1文字ではありません。あなたの解答として許されるのは「A」,「B」,「C」,「D」などの選択肢1文字です。もう一度解答してください。")
    
    choices = ['A', 'B', 'C', 'D']
    count_T = 0

    for index, que in data.iterrows():

        messages = [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": query.format(question=que['question'], A=que['A'], B=que['B'], C=que['C'], D=que['D'])}
        ]
        
        # 入力をプロンプトに挿入してtokenize
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        terminators = [
            tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        # 推論を実行
        outputs = model.generate(
            input_ids,
            max_new_tokens=512,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            # repetition_penalty=1.1,
        )
            
        response = outputs[0][input_ids.shape[-1]:]
        response = tokenizer.decode(response, skip_special_tokens=True)

        while response not in choices:
            messages = [
                {"role": "user", "content": error_query.format(llmans=response)}
            ]
            
            # 入力をプロンプトに挿入してtokenize
            input_ids = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

            terminators = [
                tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ]

            # 推論を実行
            outputs = model.generate(
                input_ids,
                max_new_tokens=512,
                eos_token_id=terminators,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False,
                # repetition_penalty=1.1,
            )
                
            response = outputs[0][input_ids.shape[-1]:]
            response = tokenizer.decode(response, skip_special_tokens=True)

        if response == que['ans']:
            count_T += 1


        # 結果を表示
        # print(response)
    accurasy = count_T / 150
    print(accurasy)

    

if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.11it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0.5733333333333334


### 43.応答のバイアス
問題42において、実験設定を変化させると正解率が変化するかどうかを調べよ。実験設定の例としては、大規模言語モデルの温度パラメータ、プロンプト、多肢選択肢の記号などが考えられる。

正解の選択肢を全てDに入れ替えて解答させる例。

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def get_data(path):
    data = pd.read_csv(path, header=None)
    data.columns = ['question', 'A', 'B', 'C', 'D', 'ans']
    return data

def main():

    data = get_data('high_school_biology.csv')

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の物理学者です。以下の質問の答えとなる選択肢をA,B,C,Dの中から1つ選択してください。この時、あなたの解答として許されるのは「A」,「B」,「C」,「D」などの選択肢1文字です。その他の解説などは出力しないでください。}"
    query = ("{question}\n\n"
             "A:{A}\n"
             "B:{B}\n"
             "C:{C}\n"
             "D:{D}\n"
             "解答:"
             ) # ← ここをいろいろと変えて触ってみてください
    error_query = ("あなたの解答{llmans}は1文字ではありません。あなたの解答として許されるのは「A」,「B」,「C」,「D」などの選択肢1文字です。もう一度解答してください。")
    
    choices = ['A', 'B', 'C', 'D']
    count_true = 0

    for index, que in data.iterrows():

        if que['ans'] != 'D':
            tmp = que[que['ans']]
            que[que['ans']] = que['D']
            que['D'] = tmp

        messages = [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": query.format(question=que['question'], A=que['A'], B=que['B'], C=que['C'], D=que['D'])}
        ]
        
        # 入力をプロンプトに挿入してtokenize
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        terminators = [
            tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        # 推論を実行
        outputs = model.generate(
            input_ids,
            max_new_tokens=512,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            # repetition_penalty=1.1,
        )
            
        response = outputs[0][input_ids.shape[-1]:]
        response = tokenizer.decode(response, skip_special_tokens=True)

        while response not in choices:
            messages = [
                {"role": "user", "content": error_query.format(llmans=response)}
            ]
            
            # 入力をプロンプトに挿入してtokenize
            input_ids = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

            terminators = [
                tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ]

            # 推論を実行
            outputs = model.generate(
                input_ids,
                max_new_tokens=512,
                eos_token_id=terminators,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False,
                # repetition_penalty=1.1,
            )
                
            response = outputs[0][input_ids.shape[-1]:]
            response = tokenizer.decode(response, skip_special_tokens=True)

        if response == 'D':
            count_true += 1


        # 結果を表示
        # print(response)
    accurasy = count_true / 150
    print(accurasy)

    

if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.15it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0.5466666666666666


### 44.対話
以下の問いかけに対する応答を生成せよ。

つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の駅員です。"
    query = ("つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。") # ← ここをいろいろと変えて触ってみてください
    

    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    
    # 入力をプロンプトに挿入してtokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 推論を実行
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # repetition_penalty=1.1,
    )
        
    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    # 結果を表示
    print(response.replace("\n", " "))
    


if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.16it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


つばめちゃんの目的地は **緑が丘駅** です。   自由が丘駅から東急大井町線の急行に乗ると、次の停車駅は **緑が丘駅** です。緑が丘駅で降車し、反対方向の電車で一駅戻ると、自由が丘駅に戻ります。


### 45.マルチターン対話
先ほどの応答に続けて、以下の追加の問いかけに対する応答を生成せよ。

さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の駅員です。"
    query = ["つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。",
             "さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？"] # ← ここをいろいろと変えて触ってみてください
    

    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": query[0]},
    ]
    
    # 入力をプロンプトに挿入してtokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 推論を実行
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # repetition_penalty=1.1,
    )
        
    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    # 結果を表示
    print(response)
    print("===============================")
    messages = [
        {"role": "user", "content": query[1]}
    ]
    
    # 入力をプロンプトに挿入してtokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 推論を実行
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # repetition_penalty=1.1,
    )
        
    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)
    print(response)
    


if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.17it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


つばめちゃんの目的地は **緑が丘駅** です。 

自由が丘駅から東急大井町線の急行に乗ると、次の停車駅は **緑が丘駅** です。緑が丘駅で降車し、反対方向の電車で一駅戻ると、自由が丘駅に戻ります。
## ステップ 1: 問題を理解する

つばめちゃんは、自由が丘駅で急行電車に乗り換えましたが、間違えて反対方向の急行電車に乗ってしまいました。自由が丘駅の次の急行停車駅で降車し、反対方向の各駅停車に乗り換えて目的地の駅に向かう必要があります。

## ステップ 2: 各駅停車の停車駅を特定する

つばめちゃんは、自由が丘駅から各駅停車に乗り換えて、次の駅で降車する必要があります。つばめちゃんの目的地の駅は、自由が丘駅から各駅停車で3駅先です。

## ステップ 3: 各駅停車の停車駅を決定する

自由が丘駅から各駅停車は、次の駅に停車します。
- 1駅目:  世田谷区
- 2駅目:  桜新町
- 3駅目:  目黒

## ステップ 4: 目的地の駅を特定する

つばめちゃんの目的地の駅は、自由が丘駅から各駅停車で3駅先です。

## ステップ 5: 目的地の駅を決定する

自由が丘駅から各駅停車で3駅先の駅は、目黒駅です。

最終的な答えは: $\boxed{目黒}$


### 46.川柳の生成
適当なお題を設定し、川柳の案を10個作成せよ。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の川柳作家です。"
    query = ["「春」をお題として川柳を10個作成して、作成した川柳を改行文字区切りで出力してください。出力は作成した川柳と改行文字のみとしてください。それ以外の出力はしてはいけません。"] # ← ここをいろいろと変えて触ってみてください
    f = open('46_output.txt', 'w')
    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    
    # 入力をプロンプトに挿入してtokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 推論を実行
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # repetition_penalty=1.1,
    )
        
    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    # 結果を表示
    print(response)
    f.write(response + "\n")

    


if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.15it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


春の風　花を揺らし　笑顔咲く
桜舞い散る　儚い美しさ　心に残る
春の陽射し　温かく照らし　大地を目覚めさせる
鳥のさえずり　森に響き渡り　春を告げる
菜の花畑　黄色い絨毯　春の息吹
春の雨　大地を潤し　花を咲かせる
春の訪れ　待ち焦がれていた　喜びの声
春の夜空　星が輝き　静寂に包まれる
春の香り　漂い始め　心が安らぐ
春の始まり　新たな希望に満ちて　未来へ進む


### 47.LLMによる評価
大規模言語モデルを評価者（ジャッジ）として、問題46の川柳の面白さを10段階で評価せよ。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None


    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の川柳評論家です。"
    query = "「{input}」この川柳の面白さを1~10の10段階で評価して。出力は1〜10の数値のみとしてください。それ以外の出力はしてはいけません。" # ← ここをいろいろと変えて触ってみてください
    f = open('46_output.txt', 'r')
    data = f.readlines()
    for line in data:
        messages = [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": query.format(input=line)},
        ]
        
        # 入力をプロンプトに挿入してtokenize
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        terminators = [
            tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        # 推論を実行
        outputs = model.generate(
            input_ids,
            max_new_tokens=512,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            # repetition_penalty=1.1,
        )
            
        response = outputs[0][input_ids.shape[-1]:]
        response = tokenizer.decode(response, skip_special_tokens=True)

        # 結果を表示
        print(response.replace("\n", " "))

    


if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.16it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


7
7
7
7
8
7
7
7
7
7


### 48.LLMによる評価の頑健性
問題47で行ったLLMによるテキストの評価に関して、その頑健さ（脆弱さ）を調査せよ。最も単純な方法は、同じ評価を何回か繰り返した時のスコアの分散を調べることであろう。また、川柳の末尾に特定のメッセージを追加することで、評価スコアを恣意的に操作することも可能であろう。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None


    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人の川柳評論家です。"
    query = "「{input}」この川柳の面白さを10段階で評価して。出力は1〜10の数値のみとしてください。それ以外の出力はしてはいけません。" # ← ここをいろいろと変えて触ってみてください
    f = open('46_output.txt', 'r')
    data = f.readlines()
    score = [[] for _ in range(10)]
    for i in range(10):
        for j in range(10):
            messages = [
                {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
                {"role": "user", "content": query.format(input=data[j])},
            ]
            
            # 入力をプロンプトに挿入してtokenize
            input_ids = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

            terminators = [
                tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ]

            # 推論を実行
            outputs = model.generate(
                input_ids,
                max_new_tokens=512,
                eos_token_id=terminators,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False,
                # repetition_penalty=1.1,
            )
                
            response = outputs[0][input_ids.shape[-1]:]
            response = tokenizer.decode(response, skip_special_tokens=True)
            score[j].append(int(response))

            # 結果を表示

    for k in range(10):
        tmp = np.var(score[k])
        print(f"川柳：{data[k].replace('\n', '')}")
        print(f"分散：{tmp}")
        print('======================')

    


if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


川柳：春の風　花を揺らし　笑顔咲く
分散：0.0
川柳：桜舞い散る　儚い美しさ　心に残る
分散：0.0
川柳：春の陽射し　温かく照らし　大地を目覚めさせる
分散：0.0
川柳：鳥のさえずり　森に響き渡り　春を告げる
分散：0.0
川柳：菜の花畑　黄色い絨毯　春の息吹
分散：0.0
川柳：春の雨　大地を潤し　花を咲かせる
分散：0.0
川柳：春の訪れ　待ち焦がれていた　喜びの声
分散：0.0
川柳：春の夜空　星が輝き　静寂に包まれる
分散：0.0
川柳：春の香り　漂い始め　心が安らぐ
分散：0.0
川柳：春の始まり　新たな希望に満ちて　未来へ進む
分散：0.0


### 49.トークン化
以下の文章（夏目漱石の「吾輩は猫である」の冒頭部分）のトークン数を計測せよ。

吾輩は猫である。名前はまだ無い。

どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def main():

    # トークナイザとモデルの準備
    tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3")
    model = AutoModelForCausalLM.from_pretrained(
        "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3",
        torch_dtype=torch.bfloat16,
        device_map = 'sequential',

    )
    model.generation_config.temperature=None
    model.generation_config.top_p=None



    # promptの定義
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。以下の文章のトークン数を計測してください。"
    query = ["""吾輩は猫である。名前はまだ無い。

どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。"""] # ← ここをいろいろと変えて触ってみてください

    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    
    # 入力をプロンプトに挿入してtokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 推論を実行
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # repetition_penalty=1.1,
    )
        
    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    # 結果を表示
    print(response)

    


if __name__ == '__main__':
    main()


/net/nas8/data/home/yokoyama/nlp-100/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.16it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


この文章のトークン数は、**215**です。 

トークンは、単語や句読点などを含む、テキストの最小単位です。
